In [36]:
import numpy as np
import pandas as pd
import xgboost
import time
import os
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
ds_train = pd.read_hdf('../data/DS_2_train.hdf')

In [34]:
ds_train.head()
ds_train[['index', 'event_id', 'signal']].head()

,index,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number
0,0,-999,66162.843750,65620.031250,0.0,0.166890,0.276674,2.988714,0.0,44
1,1,-999,43900.445312,67091.671875,0.0,-0.179967,-0.478795,1.901420,0.0,98
2,2,-999,37564.019531,26618.890625,55599.0,-0.267820,-0.282087,2.510245,0.0,64
3,3,-999,39380.027344,48135.425781,19395.0,0.265197,-0.577493,1.215219,0.0,63
4,4,-999,65040.781250,44018.468750,12930.0,-0.711161,-0.245592,2.973661,0.0,2


,index,event_id,signal
0,0,-999,0.0
1,1,-999,0.0
2,2,-999,0.0
3,3,-999,0.0
4,4,-999,0.0


In [21]:
X_full = ds_train.copy()
X_full = X_full.drop('index', axis=1).drop('event_id', axis=1).drop('signal', axis=1)

X_train = X_full[X_full.brick_number < 90].drop('brick_number', axis=1)
X_test = X_full[X_full.brick_number > 89].drop('brick_number', axis=1)
Y_train = ds_train[ds_train.brick_number < 90]['signal']
Y_test = ds_train[ds_train.brick_number > 89]['signal']


X_train.head()
X_test.head()
Y_train.head()
Y_test.head()

,X,Y,Z,TX,TY,chi2
0,66162.843750,65620.031250,0.0,0.166890,0.276674,2.988714
2,37564.019531,26618.890625,55599.0,-0.267820,-0.282087,2.510245
3,39380.027344,48135.425781,19395.0,0.265197,-0.577493,1.215219
4,65040.781250,44018.468750,12930.0,-0.711161,-0.245592,2.973661
5,55179.875000,24942.843750,41376.0,-0.173363,-0.318434,2.488240


,X,Y,Z,TX,TY,chi2
1,43900.445312,67091.671875,0.0,-0.179967,-0.478795,1.901420
13,33727.878906,55500.921875,46548.0,0.252502,0.468155,0.766934
23,30087.265625,21714.402344,6465.0,-0.493513,-0.164602,2.835666
39,70787.812500,69830.609375,31032.0,0.322359,0.117708,2.835725
49,37411.660156,56671.851562,49134.0,0.124289,-0.016574,1.369881


0    0.0
2    0.0
3    0.0
4    0.0
5    0.0
Name: signal, dtype: float64

1     0.0
13    0.0
23    0.0
39    0.0
49    1.0
Name: signal, dtype: float64

In [37]:
n_est = 8

#model = xgboost.XGBClassifier(n_estimators=n_est, nthread=8, max_depth=2)#, reg_alpha=0, reg_lambda=0.5)
#model = GradientBoostingClassifier(n_estimators=n_est, max_depth=4, loss='exponential')
model = RandomForestClassifier(n_estimators=n_est, max_depth=3, n_jobs=8)
#model = xgboost.XGBClassifier(n_estimators=n_est, nthread=8, max_depth=3)

t1= time.time()
model.fit(X_train, Y_train)
pred_train = model.predict_proba(X_train)[:,1]
pred_test = model.predict_proba(X_test)[:,1]
print 'n_est:', n_est
print 'time taken:', time.time() - t1
print 'train score:', roc_auc_score(Y_train, pred_train)
print 'test score:', roc_auc_score(Y_test, pred_test)
os.system('say "dataset validated"')

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=8, n_jobs=8, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

n_est: 8
time taken: 26.4734950066
train score: 0.906929316083
test score: 0.906485783433


0

In [45]:
for arg in [np.float64, np.float32, np.float16]:
    print np.finfo(arg).max

1.79769313486e+308
3.40282e+38
65504.0
